In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" #print everything

import warnings
warnings.filterwarnings("ignore") #ignore warnings

import os
os.chdir(r'C:\Users\Hannes Verschueren\Documents\SPRING SEMESTER 2021\Advanced Analytics\Final Assignment 2') #set working directory
os.getcwd()

In [ ]:
train=pd.read_csv('train.csv',sep=";")


In [ ]:
train.info()

In [ ]:
train.shape

In [ ]:
X_train_pp.columns.to_list()

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

corr = train.corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

plt.savefig('correlation_plot_.png')

In [ ]:
#claim_language = float64??

X_train_pp.claim_language=X_train_pp.claim_language.astype("object")
X_test_pp.claim_language=X_test_pp.claim_language.astype("object")
X_eval_pp.claim_language=X_eval_pp.claim_language.astype("object")

#claim_date_occured_day_of_the_week and claim_date_registered_day_of_the_week are int64, have to become categorical

X_train_pp.claim_date_occured_day_of_the_week=X_train_pp.claim_date_occured_day_of_the_week.astype("object")
X_test_pp.claim_date_occured_day_of_the_week=X_test_pp.claim_date_occured_day_of_the_week.astype("object")
X_eval_pp.claim_date_occured_day_of_the_week=X_eval_pp.claim_date_occured_day_of_the_week.astype("object")

X_train_pp.claim_date_registered_day_of_the_week=X_train_pp.claim_date_registered_day_of_the_week.astype("object")
X_test_pp.claim_date_registered_day_of_the_week=X_test_pp.claim_date_registered_day_of_the_week.astype("object")
X_eval_pp.claim_date_registered_day_of_the_week=X_eval_pp.claim_date_registered_day_of_the_week.astype("object")

X_train_pp.info()

In [ ]:
def deal_with_missing_values_categorical(df):
    
    numeric_columns = df.select_dtypes(include='number').columns
    categorical_columns=df.select_dtypes(include='object').columns
    df_numeric=df[numeric_columns]
    df_categorical=df[categorical_columns]
    df_rest=df.drop(numeric_columns,axis=1).drop(categorical_columns,axis=1)
    
    df_categorical=df_categorical.astype("string").fillna(value="Unknown").astype("category")
    
    df=pd.concat([df_categorical,df_numeric,df_rest],axis=1)
    
    return df

X_train_pp=deal_with_missing_values_categorical(X_train_pp)
X_test_pp=deal_with_missing_values_categorical(X_test_pp)
X_eval_pp=deal_with_missing_values_categorical(X_eval_pp)

X_train_pp.info()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

#initial weights
y_train["weights"]=y_train["claim_amount"]*y_train["fraud"]


#adjusted weights
mms=MinMaxScaler()
y_train["scaled_claim_amount"]=mms.fit_transform(y_train[["claim_amount"]])
y_train["weight_factor"]=y_train["fraud"].replace(0,0.1)
y_train["adjusted_weights"]=y_train["scaled_claim_amount"]*y_train["weight_factor"]

y_train.sort_values(by="adjusted_weights",ascending=False).head()

In [ ]:
#sklearndf

from sklearndf.transformation import (
    ColumnTransformerDF,
    OneHotEncoderDF,
    SimpleImputerDF,
    StandardScalerDF
)
from sklearndf.pipeline import PipelineDF
from imblearn.over_sampling import ADASYN

numeric_features = X_train_pp.select_dtypes(include='number').columns
categorical_features = X_train_pp.select_dtypes(include='category').columns

categorical_transformer = PipelineDF(steps=[
    ('onehot', OneHotEncoderDF(sparse=False,handle_unknown='ignore')),
])

numeric_transformer = PipelineDF(steps=[
    ('imputer', SimpleImputerDF(strategy='median')),
    ('scaler', StandardScalerDF()),
])

preprocessor = ColumnTransformerDF(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

X_train_pp_df=preprocessor.fit_transform(X_train_pp)
ada=ADASYN()
X_train_pp_df_sampled,y_train_sampled=ada.fit_sample(X_train_pp_df,y_train["fraud"])

X_train_pp_df_sampled.shape

In [ ]:
#shapley values

import shap
from xgboost import XGBClassifier

shap.initjs()

clf=XGBClassifier(colsample_bytree=0.5, gamma=0.1, learning_rate=0.05, max_depth=12, min_child_weight=1)
clf.fit(X_train_pp_df_sampled,y_train_sampled)

#https://github.com/slundberg/shap/issues/1215
mybooster = clf.get_booster()
model_bytearray = mybooster.save_raw()[4:]
def myfun(self=None):
    return model_bytearray
mybooster.save_raw = myfun

shap_values = shap.TreeExplainer(mybooster).shap_values(X_train_pp_df_sampled)
shap.summary_plot(shap_values, X_train_pp_df_sampled, plot_type="bar")

In [ ]:
#dealing with class imbalance - ADASYN vs SMOTE

import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA

from imblearn.over_sampling import ADASYN,SMOTE

# Instanciate a PCA object for the sake of easy visualisation
pca = PCA(n_components=2)
# Fit and transform x to visualise inside a 2D feature space
X_vis = pca.fit_transform(X_train_pp_df)

# ADASYN
ada = ADASYN()
X_resampled, y_resampled = ada.fit_sample(X_train_pp_df, y_train["fraud"])
X_res_vis = pca.transform(X_resampled)

# SMOTE
smo=SMOTE(k_neighbors=10)
X_resampled_smo, y_resampled_smo = smo.fit_sample(X_train_pp_df, y_train["fraud"])
X_res_vis_smo = pca.transform(X_resampled_smo)

# Three subplots, unpack the axes array immediately
f, (ax1, ax2, ax3) = plt.subplots(1, 3)

c0 = ax1.scatter(X_vis[y_train["fraud"] == 0, 0], X_vis[y_train["fraud"] == 0, 1], label="Class #0",
                 alpha=0.5)
c1 = ax1.scatter(X_vis[y_train["fraud"] == 1, 0], X_vis[y_train["fraud"] == 1, 1], label="Class #1",
                 alpha=0.5)
ax1.set_title('Original set')

ax2.scatter(X_res_vis[y_resampled == 0, 0], X_res_vis[y_resampled == 0, 1],
            label="Class #0", alpha=.5)
ax2.scatter(X_res_vis[y_resampled == 1, 0], X_res_vis[y_resampled == 1, 1],
            label="Class #1", alpha=.5)
ax2.set_title('ADASYN')

ax3.scatter(X_res_vis_smo[y_resampled_smo == 0, 0], X_res_vis_smo[y_resampled_smo == 0, 1],
            label="Class #0", alpha=.5)
ax3.scatter(X_res_vis_smo[y_resampled_smo == 1, 0], X_res_vis_smo[y_resampled_smo == 1, 1],
            label="Class #1", alpha=.5)
ax3.set_title('SMOTE')

# make nice plotting
for ax in (ax1, ax2,ax3):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()
    ax.spines['left'].set_position(('outward', 10))
    ax.spines['bottom'].set_position(('outward', 10))
    ax.set_xlim([-10, 10])
    ax.set_ylim([-6, 8])

plt.figlegend((c0, c1), ('Class #0', 'Class #1'), loc='lower center',
              ncol=2, labelspacing=0.)
plt.tight_layout(pad=3)
plt.savefig("oversampling_methods.jpg",dpi=300)
plt.show()

In [ ]:
# USEFUL FUNCTIONS

# function to evaluate performance of model on own test set

from sklearn import metrics

def evaluate_outcome(y_test,y_pred,y_pred_proba,test_id):
    
    #AUC
    fpr, tpr, thresholds = metrics.roc_curve(y_test["fraud"], y_pred, pos_label=1)
    AUC=metrics.auc(fpr, tpr)
    
    #Custom score:
    index_list=list(y_test.index)
    y_pred_df=pd.DataFrame(y_pred,columns=["y_pred"])
    y_pred_proba_df=pd.DataFrame(y_pred_proba[:,1],columns=["y_pred_proba"])
    index_df=pd.DataFrame(index_list,columns=["index_list"])
    y_pred_and_pred_proba_df=pd.concat([y_pred_proba_df,y_pred_df,index_df],axis=1).set_index("index_list")
    to_evaluate=pd.concat([test_id,y_test,y_pred_and_pred_proba_df],axis=1)
    
    y_pred_proba_df=pd.DataFrame(y_pred_proba[:,0],columns=["y_pred_proba"])
    to_evaluate=to_evaluate.sort_values(by=["y_pred_proba"],ascending=False)
    to_evaluate_top_100=to_evaluate.head(100)
    
    score=0
    for i in range(len(to_evaluate_top_100.fraud)):
        if to_evaluate_top_100.fraud.iloc[i] == to_evaluate_top_100.y_pred.iloc[i]:
            score+= to_evaluate_top_100.claim_amount.iloc[i]
    
    print("AUC score of the model: " + str(np.round(AUC,decimals=4))), print("Custom score:"+ str(np.round(score,decimals=2)))
    

# leaderboard predictions to proper format

def predictions_to_csv(index,y_pred_proba,model:str):
    y_pred_proba_df=pd.DataFrame(y_pred_proba[:,1],columns=["PROB"])
    output=pd.concat([index.rename(columns={"claim_id":"ID"},inplace=False).astype("int64"),y_pred_proba_df],axis=1)
    output.to_csv("predictions-"+model+".csv",sep=",",index=False)
    return output.head(),print("Predictions saved to csv file :-)")



In [ ]:
X_train_pp.info()

In [ ]:
# MODEL : Logistic regression (baseline)

from imblearn.pipeline import Pipeline 
from imblearn.over_sampling import SMOTE, ADASYN #oversampling methods
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV


numeric_features = X_train_pp.select_dtypes(include='number').columns
categorical_features = X_train_pp.select_dtypes(include='category').columns

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
])

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

pipe = Pipeline([
     ('preprocessor', preprocessor),
     ('clf', LogisticRegression())
])

pipe.fit(X_train_pp,y_train["fraud"])#,clf__sample_weight=y_train["adjusted_weights"])

#own test set predictions
y_pred=pipe.predict(X_test_pp)
y_pred_proba=pipe.predict_proba(X_test_pp)

#evaluation set predictions
y_pred_proba_eval=pipe.predict_proba(X_eval_pp)

In [ ]:
#evaluate + save predictions to file -- Logistic Regression (baseline)
evaluate_outcome(y_test,y_pred,y_pred_proba,test_id)
#predictions_to_csv(eval_id,y_pred_proba_eval,"logisticregression(baseline)")

In [ ]:
## evaluate feature importance

#to get the encodes features
categorical_enc_features=pipe.named_steps["preprocessor"].transformers_[1][1].named_steps['onehot'].get_feature_names(categorical_features).tolist()
numerical_features=X_train_pp.select_dtypes(include='number').columns.tolist()
features_enc=categorical_enc_features+numerical_features

#to get the coefficients
importance=pipe.named_steps["clf"].coef_[0]

#descriptive statistics of the coefficients
from scipy import stats
min_feature_importance=stats.describe(importance).minmax[0]
max_feature_importance=stats.describe(importance).minmax[1]
mean_feature_importance=stats.describe(importance).mean
stddev_feature_importance=np.sqrt(stats.describe(importance).variance)

#define boundaries for importance features
upper_lower=max_feature_importance-2*stddev_feature_importance
lower_upper=min_feature_importance+2*stddev_feature_importance

#enumerate features considered to be important
important_features=[]
coefficients=[]
for index,value in enumerate(importance):
    if (value>upper_lower) or (value<lower_upper):
        print(features_enc[index]+": "+str(value))
        important_feature=[features_enc[index]]
        important_features=important_features+important_feature
        coefficient=[value]
        coefficients=coefficients+coefficient

df_feature_importance=pd.DataFrame(list(zip(important_features, coefficients)),columns=["important_features","coefficients"])

sns.reset_orig()
sns.set_style("whitegrid")
fig_dims = (40,20)
fig, ax = plt.subplots(figsize=fig_dims)
importance_graph=sns.barplot(x=df_feature_importance.important_features,y=df_feature_importance.coefficients,ax=ax)
importance_graph.set_xticklabels(labels=importance_graph.get_xticklabels(),rotation=90,fontsize=20)
importance_graph.set_xlabel("Features",fontsize=20)
importance_graph.set_ylabel("Importance",fontsize=20)
importance_graph.set_title("Most important features",fontsize=40,y=1.1)

plt.savefig("feature_importance-logisticregression.jpeg",bbox_inches = "tight")

In [ ]:
# MODEL : Logistic regression + ADASYN

from imblearn.pipeline import Pipeline 
from imblearn.over_sampling import SMOTE, ADASYN #oversampling methods
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV


numeric_features = X_train_pp.select_dtypes(include='number').columns
categorical_features = X_train_pp.select_dtypes(include='category').columns

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
])

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

pipe = Pipeline([
     ('preprocessor', preprocessor),
     ('clf', LogisticRegression())
])

pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('sampling', ADASYN(n_neighbors=10,random_state=7)),
        ('clf', LogisticRegression(penalty="l2",solver="saga"))
    ])

param_grid = {
    'sampling__n_neighbors': [1, 2, 3, 5, 10],
    'clf__penalty': ["l1", "l2", "elasticnet"],
}
search = GridSearchCV(pipe, param_grid, n_jobs=-1,verbose=2,scoring="average_precision")
#search.fit(X_train_pp, y_train["fraud"])
#print("Best parameters: ", search.best_params_)

pipe.fit(X_train_pp,y_train["fraud"])

#own test set predictions
y_pred=pipe.predict(X_test_pp)
y_pred_proba=pipe.predict_proba(X_test_pp)

#evaluation set predictions
y_pred_proba_eval=pipe.predict_proba(X_eval_pp)

In [ ]:
#evaluate + save predictions to file -- Logistic Regression + ADASYN
evaluate_outcome(y_test,y_pred,y_pred_proba,test_id)
predictions_to_csv(eval_id,y_pred_proba_eval,"logisticregression+ADASYN")

In [ ]:
# MODEL: XGBOOST + ADASYN (baseline)

from imblearn.pipeline import Pipeline 
from imblearn.over_sampling import SMOTE, ADASYN #oversampling methods
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV


numeric_features = X_train_pp.select_dtypes(include='number').columns
categorical_features = X_train_pp.select_dtypes(include='category').columns

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
])

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)


pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('sampling', ADASYN(random_state=7)),
        ('clf', XGBClassifier())
    ])

param_grid = {
    "sampling__n_neighbors": [1, 2, 3, 5, 10],
    "clf__learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
    "clf__max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
    "clf__min_child_weight" : [ 1, 3, 5, 7 ],
    "clf__gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
    "clf__colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ] 
}
#search = GridSearchCV(pipe, param_grid, n_jobs=-1,verbose=2,scoring="recall",cv=3)
#search.fit(X_train_pp, y_train["fraud"])
#print("Best parameters: ", search.best_params_)

pipe.fit(X_train_pp,y_train["fraud"])

#own test set predictions
y_pred=pipe.predict(X_test_pp)
y_pred_proba=pipe.predict_proba(X_test_pp)

#evaluation set predictions
y_pred_proba_eval=pipe.predict_proba(X_eval_pp)

In [ ]:
#evaluate + save predictions to file -- XGBOOST + ADASYN (baseline)
evaluate_outcome(y_test,y_pred,y_pred_proba,test_id)
predictions_to_csv(eval_id,y_pred_proba_eval,"XGBOOST+ADASYN(baseline)")

In [ ]:
# MODEL: XGBOOST + ADASYN + CV (scoring=roc_auc)

from imblearn.pipeline import Pipeline 
from imblearn.over_sampling import SMOTE, ADASYN #oversampling methods
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV


numeric_features = X_train_pp.select_dtypes(include='number').columns
categorical_features = X_train_pp.select_dtypes(include='category').columns

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
])

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)


pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('sampling', ADASYN(n_neighbors=1,random_state=7)),
        ('clf', XGBClassifier(colsample_bytree=0.5, gamma=0.1, learning_rate=0.05, max_depth=12, min_child_weight=1)) #optimal parameters GridSearchCV
    ])

param_grid = {
    "sampling__n_neighbors": [1, 2, 3, 5, 10],
    "clf__learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
    "clf__max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
    "clf__min_child_weight" : [ 1, 3, 5, 7 ],
    "clf__gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
    "clf__colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ] 
}
#search = GridSearchCV(pipe, param_grid, n_jobs=-1,verbose=2,scoring="recall",cv=3)
#search.fit(X_train_pp, y_train["fraud"])
#print("Best parameters: ", search.best_params_)

pipe.fit(X_train_pp,y_train["fraud"])

#own test set predictions
y_pred=pipe.predict(X_test_pp)
y_pred_proba=pipe.predict_proba(X_test_pp)

#evaluation set predictions
y_pred_proba_eval=pipe.predict_proba(X_eval_pp)

In [ ]:
#evaluate + save predictions to file -- XGBOOST + ADASYN + CV(roc_auc)
evaluate_outcome(y_test,y_pred,y_pred_proba,test_id)
predictions_to_csv(eval_id,y_pred_proba_eval,"XGBoost+ADASYN+CV(roc_auc)")

In [ ]:
#to get sample weights
y_train["fraud"].value_counts()
num_fraud=205
num_not_fraud=36770
pos_weight=num_not_fraud/num_fraud
pos_weight

In [ ]:
# MODEL: XGBOOST + sample weights (scoring=roc_auc)

from imblearn.pipeline import Pipeline 
from imblearn.over_sampling import SMOTE, ADASYN #oversampling methods
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV


numeric_features = X_train_pp.select_dtypes(include='number').columns
categorical_features = X_train_pp.select_dtypes(include='category').columns

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
])

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)


pipe = Pipeline([
        ('preprocessor', preprocessor),
        #('sampling', ADASYN(n_neighbors=1,random_state=7)),
        ('clf', XGBClassifier(colsample_bytree=0.5, gamma=0.1, learning_rate=0.05, max_depth=12, min_child_weight=1,scale_pos_weight=pos_weight)) #optimal parameters GridSearchCV
    ])

param_grid = {
    "sampling__n_neighbors": [1, 2, 3, 5, 10],
    "clf__learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
    "clf__max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
    "clf__min_child_weight" : [ 1, 3, 5, 7 ],
    "clf__gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
    "clf__colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ] 
}
#search = GridSearchCV(pipe, param_grid, n_jobs=-1,verbose=2,scoring="recall",cv=3)
#search.fit(X_train_pp, y_train["fraud"])
#print("Best parameters: ", search.best_params_)

pipe.fit(X_train_pp,y_train["fraud"],clf__sample_weight=y_train["adjusted_weights"])

#own test set predictions
y_pred=pipe.predict(X_test_pp)
y_pred_proba=pipe.predict_proba(X_test_pp)

#evaluation set predictions
y_pred_proba_eval=pipe.predict_proba(X_eval_pp)

In [ ]:
#evaluate + save predictions to file -- XGBOOST + sample_weights
evaluate_outcome(y_test,y_pred,y_pred_proba,test_id)
predictions_to_csv(eval_id,y_pred_proba_eval,"XGBoost+sample_weights")

In [ ]:
#RandomSearchCV on other computer with more cores
print("Best parameters:  {'clf__min_child_weight': 7, 'clf__max_depth': 8, 'clf__learning_rate': 0.05, 'clf__gamma': 0.3, 'clf__colsample_bytree': 0.5}")

In [ ]:
# MODEL: XGBOOST + PF

from imblearn.pipeline import Pipeline 
from imblearn.over_sampling import SMOTE, ADASYN #oversampling methods
from sklearn.preprocessing import OneHotEncoder,PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV


numeric_features = X_train_pp.select_dtypes(include='number').columns
categorical_features = X_train_pp.select_dtypes(include='category').columns

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
])

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)


pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('poly',PolynomialFeatures(degree=2,interaction_only=True)),
        ('sampling', ADASYN(n_neighbors=1,random_state=7)),
        ('clf', XGBClassifier(colsample_bytree=0.5, gamma=0.3, learning_rate=0.05, max_depth=8, min_child_weight=7)) #optimal parameters GridSearchCV
    ])

param_grid = {
    "sampling__n_neighbors": [1, 5, 10],
    "clf__learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
    "clf__max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
    "clf__min_child_weight" : [ 1, 3, 5, 7 ],
    "clf__gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
    "clf__colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ] 
}
#search = GridSearchCV(pipe, param_grid, n_jobs=-1,verbose=2,scoring=‘roc_auc’,cv=3)
#search.fit(X_train_pp, y_train["fraud"])
#print("Best parameters: ", search.best_params_)

pipe.fit(X_train_pp,y_train["fraud"])

#own test set predictions
y_pred=pipe.predict(X_test_pp)
y_pred_proba=pipe.predict_proba(X_test_pp)

#evaluation set predictions
y_pred_proba_eval=pipe.predict_proba(X_eval_pp)

In [ ]:
#evaluate + save predictions to file -- XGBOOST + sample_weights
evaluate_outcome(y_test,y_pred,y_pred_proba,test_id)
predictions_to_csv(eval_id,y_pred_proba_eval,"XGBoost+PF+RandomSearchCV(roc_auc)")

In [ ]:
X_train_pp.select_dtypes(include='number').columns.tolist()
